In [ ]:
#@title # Packages & Tools
#packages
!pip install texttable
!pip install openpyxl
!pip install uuid
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import os
from texttable import Texttable
#id generatiom
import uuid
# ignored warning
import warnings
warnings.filterwarnings('ignore')

#-------------------------------------------------------------------------------refined on 5/8/2024
# Tools
# file check, platform
def file_exists(file_path):
    return os.path.isfile(file_path)
def check_paths_valid(paths):
    return all(file_exists(path) for path in paths)
def order_form_platform(df):
    if df.columns[2] == 'ORDER FORM':
        return 'spe'
    elif df.columns[2] == 'SAMPLE FORM':
        return 'sample'
    else:
        return 'None'
# time now
def time_now():
    now_utc = datetime.utcnow()
    time_zone = pytz.timezone('America/New_York')
    local_time = now_utc.replace(tzinfo=pytz.utc).astimezone(time_zone)
    return local_time.strftime("%Y-%m-%d-%H:%M:%S")
# today
def date_now():
    now_utc = datetime.utcnow()
    time_zone = pytz.timezone('America/New_York')
    local_time = now_utc.replace(tzinfo=pytz.utc).astimezone(time_zone)
    return local_time.strftime("%Y-%m-%d")
# order-id
def date_now_id():
    now_utc = datetime.utcnow()
    time_zone = pytz.timezone('America/New_York')
    local_time = now_utc.replace(tzinfo=pytz.utc).astimezone(time_zone)
    return local_time.strftime("%m%d%y")
def generate_order_id(order_type):
    unique_str = uuid.uuid4().hex  # Generates a random UUID.
    if order_type == 'spe':
        return f"{'O'}-{date_now_id()}-{unique_str[:4]}"
    elif order_type == 'sample':
        return f"{'S'}-{date_now_id()}-{unique_str[:4]}"


In [10]:
#@title # File Paths (Amazon, Shopify, Walmart, DSCO, DATATRANS (SW & TSC), Order & Sample Forms, Inventory list)
#-------------------------------------------------------------------------------refined on 5/8/2024
# Uploading file paths (Shopify, Walmart, Amazon, DSCO)
shopify_path = '/content/Shopify_Order.csv'                                     # orders_export(..).csv
walmart_path = '/content/Walmart_Order.xlsx'                                    # PO_Data_...xlsx
amazon_path = '/content/Amazon_Unshipped_Orders.txt'                            # 857..txt
dsco_path =  '/content/DSCO_Order.xls'                                          # Dsco..xls

# Uploading DATATRANS file (Tractor Supply, Sportsman's Warehouse Invoice)
tsc_path = '/content/Tractor_Supply.csv'                                        # EDI invoice 2.0.csv
sw_path  = '/content/Sportsman_Warehouse.csv'                                   # EDI invoice 2.0.csv

# Uploading Order & Sample Forms in a list
OS_paths = ["/content/Order_Form.xlsx"]   #["path","",""]  # Order form/ Sample form

# Uploading inventory file
inven_path ='/content/inventory_080924.xlsx'                                    # Inventory_...xlsx

In [11]:
#@title # Order Data Transformation, SKUs Inspection, Order List Generator
#-------------------------------------------------------------------------------refined on 4/26/2024
dataframes = []
#-------------------------------------------------------------------------------refined on 4/10/2024
# Loading Inventory Data
if file_exists(inven_path):
    inven_all = pd.read_excel(inven_path)
    inven = inven_all[inven_all['USE'] == 1] # use
    inven_0 = inven_all[inven_all['USE'] == 0] # not use

#-------------------------------------------------------------------------------refined on 4/10/2024
# Data Transformation for Amazon, Shopify, Walmart, DSCO order list
if file_exists(amazon_path):
    amazon = pd.read_csv(amazon_path, delimiter='\t')
    new_amazon = amazon[['order-id','sku','product-name','quantity-to-ship','recipient-name']]
    new_amazon['platform'] = 'amazon'
    new_amazon['upc'] = ''
    cols = ['platform'] + ['order-id','sku','upc','product-name','quantity-to-ship','recipient-name']
    new_amazon = new_amazon[cols]
    dataframes.append(new_amazon)
    #display(new_amazon)

if file_exists(dsco_path):
    dsco = pd.read_excel(dsco_path, engine='xlrd')
    new_dsco = dsco[['po_number', 'line_item_sku','line_item_ean','line_item_title','line_item_quantity','ship_first_name','ship_last_name']]
    new_dsco.rename(columns = {'po_number':'order-id', 'line_item_sku':'sku','line_item_ean':'upc','line_item_title':'product-name','line_item_quantity':'quantity-to-ship'}, inplace = True)
    new_dsco['recipient-name'] = new_dsco['ship_first_name'] + ' ' + new_dsco['ship_last_name']
    new_dsco.drop(['ship_first_name', 'ship_last_name'],axis = 1, inplace = True)
    new_dsco['platform'] = 'dsco'
    new_dsco = new_dsco[['platform'] + ['order-id','sku','upc','product-name','quantity-to-ship','recipient-name']]
    new_dsco['upc'] = new_dsco['upc'].apply(lambda x :'{:.0f}'.format(x))
    dataframes.append(new_dsco)
    #display(new_dsco)

if file_exists(shopify_path):
    shopify = pd.read_csv(shopify_path)
    shopify = shopify[['Name','Lineitem sku','Lineitem name','Lineitem quantity','Shipping Name']]
    new_shopify = shopify.rename(columns = {'Name':'order-id','Lineitem sku':'sku','Lineitem name':'product-name','Lineitem quantity':'quantity-to-ship','Shipping Name':'recipient-name'})
    new_shopify['platform'] = 'shopify'
    new_shopify['upc'] = ''
    cols = ['platform'] + ['order-id','sku','upc','product-name','quantity-to-ship','recipient-name']
    new_shopify = new_shopify[cols]
    new_shopify['recipient-name'] = new_shopify['recipient-name'].fillna(method='ffill')
    dataframes.append(new_shopify)
    #display(new_shopify)

if file_exists(walmart_path):
    walmart = pd.read_excel(walmart_path, engine='openpyxl')
    walmart = walmart[['PO#','SKU','Item Description','Qty','Customer Name']]
    new_walmart = walmart.rename(columns = {'PO#':'order-id','SKU':'sku','Item Description':'product-name','Qty':'quantity-to-ship', 'Customer Name':'recipient-name'})
    new_walmart['platform'] = 'walmart'
    new_walmart['upc'] = ''
    cols = ['platform'] + ['order-id','sku','upc','product-name','quantity-to-ship','recipient-name']
    new_walmart = new_walmart[cols]
    dataframes.append(new_walmart)
    #display(new_walmart)

#-------------------------------------------------------------------------------refined on 4/26/2024
# Data Transformation for Tractor Supply
if file_exists(tsc_path):
    tsc = pd.read_csv(tsc_path)
    header_df = pd.DataFrame([tsc.columns], columns=tsc.columns)
    tsc = pd.concat([header_df, tsc], ignore_index=True)
    tsc.columns = range(tsc.shape[1])
    columns_to_drop = list(range(5, 36))
    tsc = tsc.drop(columns=columns_to_drop)
    tsc.columns = range(tsc.shape[1])

    mess_header_index = tsc.index[tsc[0] == 'mess_header'].tolist()
    start_indices = mess_header_index[0::2]
    end_indices = tsc.index[tsc[0] == 'summary'].tolist() + [len(tsc)]

    tsc_dataframes = []

    for i in range(len(start_indices)):
        start_idx = start_indices[i]
        end_idx = next(x for x in end_indices if x > start_idx)
        sub_df = tsc.iloc[start_idx:end_idx+1]
        df_filtered = sub_df[~((sub_df[0] == 'detail') & sub_df.loc[:, '1':].isna().all(axis=1))]
        df_filtered = df_filtered.reset_index(drop=True)
        tsc_dataframes.append(df_filtered)

    new_tsc_dataframes = []

    column_names = ['platform','order-id','sku','upc','product-name','quantity-to-ship','recipient-name']

    for df in tsc_dataframes:
        id = df.loc[1, 5]     # id
        name = df.loc[6, 2]   # name
        if len(df) <= 14:     # the order id may contain only one type of product.
            new_df = pd.DataFrame(columns=column_names)
            new_row1 = pd.DataFrame({'platform': 'tsc', 'order-id': [id], 'sku': [df.loc[10,3]], 'upc': [df.loc[9,3]], 'product-name':[df.loc[12,3]] ,'quantity-to-ship': [df.loc[8, 3]], 'recipient-name': name})
            new_df = pd.concat([new_df, new_row1], ignore_index=True) # creating a new DataFrame
            new_tsc_dataframes.append(new_df)
        elif len(df) > 18:
            i = 8
            multi_orders = pd.DataFrame(columns=column_names)
            while i + 5 <= len(df) - 1:
                new_row2 = pd.DataFrame({'platform': 'tsc', 'order-id': [id], 'sku': [df.loc[i+2,3]], 'upc': [df.loc[i+1,3]], 'product-name':[df.loc[i+4,3]] ,'quantity-to-ship': [df.loc[i, 3]],'recipient-name': name})
                multi_orders = pd.concat([multi_orders, new_row2], ignore_index=True)
                i = i + 5
            new_tsc_dataframes.append(multi_orders)

    tsc_list = pd.concat(new_tsc_dataframes, axis=0, ignore_index=True)
    tsc_list['order-id'] = tsc_list['order-id'].astype(int)
    dataframes.append(tsc_list)

#-------------------------------------------------------------------------------refined on 4/26/2024
# Data Transformation for Sportsman's Warehouse
if file_exists(sw_path):
    sw = pd.read_csv(sw_path)
    sw_header_df = pd.DataFrame([sw.columns], columns=sw.columns)
    sw = pd.concat([sw_header_df, sw], ignore_index=True)
    sw.columns = range(sw.shape[1])
    columns_to_drop = list(range(5, 36))
    sw = sw.drop(columns=columns_to_drop)
    sw.columns = range(sw.shape[1])

    sw_mess_header_index = sw.index[sw[0] == 'mess_header'].tolist()
    sw_start_indices = sw_mess_header_index[0::2]
    sw_end_indices = sw.index[sw[0] == 'summary'].tolist() + [len(sw)]

    sw_dataframes = []

    for i in range(len(sw_start_indices)):
        start_idx = sw_start_indices[i]
        end_idx = next(x for x in sw_end_indices if x > start_idx)
        sub_df = sw.iloc[start_idx:end_idx+1]
        df_filtered = sub_df[~((sub_df[0] == 'detail') & sub_df.loc[:, '1':].isna().all(axis=1))]
        df_filtered = df_filtered.reset_index(drop=True)
        sw_dataframes.append(df_filtered)

    # Updating missing UPCs
    sw_upc_dataframes = []
    for df in sw_dataframes:
        k = 12
        while k <= len(df):
            if df.loc[k, 0] != 'parts' and df.loc[k, 0] == 'description':
                new_upc_row = pd.DataFrame({key: [np.nan] for key in df.columns}, index=[(k-1)+0.5])
                df = pd.concat([df.iloc[:k], new_upc_row, df.iloc[k:]]).reset_index(drop=True)
            k = k + 5
        sw_upc_dataframes.append(df)

    new_sw_dataframes = []

    column_names = ['platform','order-id','sku','upc','product-name','quantity-to-ship','recipient-name']

    for df in sw_upc_dataframes:
        id = df.loc[1, 5]     # id
        name = df.loc[7, 2]   # name
        if len(df) <= 15:     # the order id may contain only one type of product.
                new_df = pd.DataFrame(columns=column_names)
                new_row1 = pd.DataFrame({'platform': 'sw', 'order-id': [id], 'sku': [df.loc[11,3]], 'upc': [df.loc[12,3]], 'product-name':[df.loc[13,3]] ,'quantity-to-ship': [df.loc[9, 3]], 'recipient-name': name})
                new_df = pd.concat([new_df, new_row1], ignore_index=True) # creating a new DataFrame
                new_sw_dataframes.append(new_df)
        elif len(df) > 18:
            i = 9
            multi_orders = pd.DataFrame(columns=column_names)
            while i + 5 <= len(df) - 1:
                new_row2 = pd.DataFrame({'platform': 'sw', 'order-id': [id], 'sku': [df.loc[i+2,3]], 'upc': [df.loc[i+3,3]], 'product-name':[df.loc[i+4,3]] ,'quantity-to-ship': [df.loc[i, 3]],'recipient-name': name})
                multi_orders = pd.concat([multi_orders, new_row2], ignore_index=True)
                i = i + 5
            new_sw_dataframes.append(multi_orders)

    sw_list = pd.concat(new_sw_dataframes, axis=0, ignore_index=True)
    sw_list['order-id'] = sw_list['order-id'].astype(int)
    #display(sw_list)
    dataframes.append(sw_list)

#-------------------------------------------------------------------------------refined on 5/8/2024
# Data Transformation for Order & Sample Forms

# Order & Sample forms
if check_paths_valid(OS_paths):
    final_new_spe = []
    final_unmatched_rows = []

    for path in OS_paths:
        spe = pd.read_excel(path)
        # display(spe)
        # order header: platform
        platform = order_form_platform(spe)
        spe.columns = range(spe.shape[1])
        ### display here only
                      # order header
        new_columns = ['platform','order-no','upload-date','order-date','sales-person','s-email',
                      # billing info
                      'billing-company', 'billing-address','billing-city', 'billing-state','billing-zip','billing-phone', 'billing-name','billing-email','billing-po',
                      # shipping info
                      'recipient-company', 'recipient-address','recipient-city','recipient-state', 'recipient-zip','recipient-phone','recipient-name','recipient-email', 'order-remark',
                      # order condition
                      'start-date','cancel-date','shipping-method','term',
                      # payment info
                      'payment-name','payment-card','payment-exp','ccv',
                      # order info
                      'style_name', 'style_no','SKU','style_color', 'size', 'q']
        # general information
        #####################order header#######################
        #platform = 'spe'
        sales_person = spe.loc[1, 13]
        now = time_now()
        order_no = generate_order_id(platform)
        order_date = spe.loc[0, 13]
        s_email = spe.loc[2, 13]
        ######################billing info######################
        bcomp = spe.loc[6, 1]
        baddress = spe.loc[7, 1]
        bcity = spe.loc[8, 1]
        bstate = spe.loc[9,1]
        bzip = spe.loc[9, 2]
        bphone = spe.loc[10, 1]
        bname = spe.loc[11, 1]
        bemail = spe.loc[12,1]
        bpo = spe.loc[13,1]
        #######################shipping info#####################
        scomp = spe.loc[6, 4]
        saddress = spe.loc[7, 4]
        scity = spe.loc[8,4]
        sstate = spe.loc[9,4]
        szip = spe.loc[9, 11]
        sphone = spe.loc[10, 4]
        sname = spe.loc[11, 4]
        semail = spe.loc[12,4]
        remark = spe.loc[13,4]
        #######################order condition###################
        start_date = spe.loc[14, 1]
        cancel_date = spe.loc[14,4]
        ship_via = spe.loc[15,1]
        term = spe.loc[15,4]
        #######################payment info######################
        cc_name = spe.loc[16,1]
        cc_number = spe.loc[16,4]
        exp = spe.loc[17,1]
        ccv = spe.loc[17,4]
        #######################order info #######################
        spes = []

        a = range(19, spe.index[-1])
        b = range(4, 13)
        for i in a:
            style_name = spe.loc[i, 0]
            style_no = spe.loc[i, 1]
            style_color = spe.loc[i, 3]
            for s in b:
                if pd.isnull(spe.loc[i, s]) == True:
                    s = s + 1
                else:
                    size = spe.loc[18, s]
                    q = spe.loc[i, s]
                    sub_spe = pd.DataFrame({'platform' : [platform],'order-id': [order_no],'upload-date':[now],'order-date':[order_date],'sales-person': [sales_person],'s-email':[s_email],
                      # billing info
                      'billing-company': [bcomp], 'billing-address':[baddress],'billing-city':[bcity], 'billing-state':[bstate],'billing-zip':[bzip],'billing-phone':[bphone], 'billing-name':[bname],'billing-email':[bemail],'billing-po':[bpo],
                      # shipping info
                      'recipient-company':[scomp], 'recipient-address':[saddress],'recipient-city':[scity],'recipient-state':[sstate], 'recipient-zip':[szip],'recipient-phone':[sphone],'recipient-name':[sname],'recipient-email':[semail], 'order-remark':[remark],
                      # order condition
                      'start-date':[start_date],'cancel-date':[cancel_date],'shipping-method':[ship_via],'term':[term],
                      # payment info
                      'payment-name':[cc_name],'payment-card':[cc_number],'payment-exp':[exp],'ccv':[ccv],
                      # order info
                      'style_name':[style_name], 'style_no':[style_no],'SKU':[None],'upc':'','style_color':[style_color], 'size':[size], 'q':[q]}) # May add a UPC later
                    spes.append(sub_spe)
                    s = s + 1
        new_spe = pd.concat(spes, axis=0)

        ###
        new_spe = new_spe.reset_index(drop = True)

        new_spe['billing-zip'] = new_spe['billing-zip'].astype(int)
        new_spe['style_no'] = new_spe['style_no'].astype(str)

        new_spe['style_name'] = new_spe['style_name'].str.upper()
        new_spe['style_no'] = new_spe['style_no'].str.upper()
        new_spe['size'] = new_spe['size'].str.upper()
        new_spe['style_color'] = new_spe['style_color'].str.lower()
        #new_spe

        ###
        # Match based on style_no, first three characters of style_color, and size
        def match_and_attach_sku(new_spe, inven):
            unmatched_rows = []
            for idx, row in new_spe.iterrows():
                # Match first three characters of the color
                color_substr = row['style_color'][:3].lower()
                mask = (inven['STYLE#'] == row['style_no']) & \
                      (inven['COLOR'].str.lower().str.contains(color_substr)) & \
                      (inven['SIZE'] == row['size'])
                matching_skus = inven[mask]['SKU']
                if not matching_skus.empty:
                    new_spe.at[idx, 'SKU'] = matching_skus.iloc[0]
                else:
                    unmatched_rows.append(row.to_dict())
            return new_spe, unmatched_rows
        ###
        new_spe, unmatched_rows = match_and_attach_sku(new_spe, inven)
        unmatched_df = pd.DataFrame(unmatched_rows)
        #display(unmatched_df)

        final_new_spe.append(new_spe)
        final_unmatched_rows.append(unmatched_df)

    final_spe = pd.concat(final_new_spe, axis = 0)
    final_spe = final_spe.reset_index(drop = True)
    final_unmatched = pd.concat(final_unmatched_rows, axis = 0)
    final_unmatched = final_unmatched.reset_index(drop = True)
    #print(final_unmatched.empty)
    ###
    if not final_unmatched.empty:
        print("\nUnmatched Data: style no, style color or size can not be matched!")
        display(final_unmatched[['platform','order-id','style_name', 'style_no','SKU','style_color','size','q']])
    else:
        print("\nOrder Result:")
        # change to the uniform version
        uniform_final_spe = final_spe[['platform','order-id','SKU','upc','style_name','q','billing-company']]
        uniform_final_spe.rename(columns = {'platform':'platform','order-id':'order-id','SKU':'sku','upc':'upc','style_name':'product-name','q':'quantity-to-ship','billing-company':'recipient-name'}, inplace = True)
        dataframes.append(uniform_final_spe)
        display(uniform_final_spe)
else:
    print('The order & sample path list is empty or not valid.')

#-------------------------------------------------------------------------------refined on 4/10/2024
# Merge DataFrames into one
if dataframes:
    order_list = pd.concat(dataframes, axis=0)
else:
    order_list = pd.DataFrame()

#-------------------------------------------------------------------------------refined on 4/10/2024
# Generated Order List
#display(order_list)
order_list['order-id'] = order_list['order-id'].astype(str)
order_list['sku'] = order_list['sku'].astype(str)
order_list['sku'] = order_list['sku'].replace({'NaN': 'null', '': 'null', 'nan':'null'})
order_list['upc'] = order_list['upc'].astype(str)
order_list['upc'] = order_list['upc'].replace({'NaN': 'null', '': 'null', 'nan':'null'})
order_list['quantity-to-ship'] = pd.to_numeric(order_list['quantity-to-ship'], errors='coerce')
order_list['recipient-name'] = order_list['recipient-name'].astype(str)
order_list = order_list.reset_index(drop = True)
#display(order_list)
#file_name_tsc = f"tsc300_.xlsx"
#order_list.to_excel(file_name_tsc, index = True)
#-------------------------------------------------------------------------------refined on 4/20/2024
# Checking order SKUs
################################################################################
unmatched_skus = order_list[~order_list['sku'].isin(inven['SKU'])][['platform','order-id','sku','upc','product-name']]

# Check if there are any such SKUs and output them
if not unmatched_skus.empty:
    print("Warning: The following SKUs from the order today do not exist in our inventory, please update SKUs from both platform and order sheets.")
    display(unmatched_skus)
    if input("Do you want to correct the SKUs now? (y/n): ").lower() != 'y':
        print("Exiting the SKU correction process.")
    else:
        for index in unmatched_skus.index:
            print('\n\n\n##################################################################################')
            print('##################################################################################')
            print('\nWrong record with index: ', index)
            display(unmatched_skus.loc[index:index])

            print('\n##################################################################################')
            print('\nSuggestion 1 SKU first part as Style#:')
            sku_prefix = unmatched_skus.loc[index:index]['sku'].values[0].split('-')[0]
            display(inven[inven['STYLE#'] == sku_prefix])
            print('\nSuggestion 2 SKU as Style#:')
            sku_prefix = unmatched_skus.loc[index:index]['sku'].values[0]
            display(inven[inven['STYLE#'] == sku_prefix])
            print('\nSuggestion 3 UPC:')
            upc_prefix = unmatched_skus.loc[index:index]['upc'].values[0]
            upc_prefix= pd.to_numeric(upc_prefix, errors='coerce')
            display(inven[inven['UPC'] == upc_prefix])
            print('##################################################################################\n')

            while True:
                new_sku = input("Please enter the correct SKU base on above information: ").strip().upper()
                if new_sku in list(inven['SKU']):
                    order_list.at[index, 'sku'] = new_sku
                    print('SKU updated!')
                    break
                else:
                    print("The SKU does not exist in our inventory. Please try again.")
        print('\nOrder list updated!')
        display(order_list)

else:
    print("All SKUs from the order today are present in the inventory!")



Order Result:


,platform,order-id,sku,upc,product-name,quantity-to-ship,recipient-name
0,spe,O-081224-74a8,SP20M079-CAMM,,KEYA,3,HUSSEY STORE
1,spe,O-081224-74a8,SP20M079-CAML,,KEYA,3,HUSSEY STORE
2,spe,O-081224-74a8,SP20M079-CAMXL,,KEYA,3,HUSSEY STORE
3,spe,O-081224-74a8,SP20M079-MOSM,,KEYA,3,HUSSEY STORE
4,spe,O-081224-74a8,SP20M079-MOSL,,KEYA,3,HUSSEY STORE
5,spe,O-081224-74a8,SP20M079-MOSXL,,KEYA,3,HUSSEY STORE
6,spe,O-081224-74a8,SS25-002KHAML,,CRILLON,4,HUSSEY STORE
7,spe,O-081224-74a8,SS25-002KHALXL,,CRILLON,4,HUSSEY STORE


,platform,order-id,sku,upc,product-name
4,amazon,111-9337684-4922606,SP21M0042N-BEIL-new,null,KANUT SPORTS Cimarron Safari Hat for Men and W...
5,amazon,111-9337684-4922606,SP21M0042N-LGM-new,null,KANUT SPORTS Cimarron Safari Hat for Men and W...
7,dsco,88416645,SS22M-08,6932542845372,CLASSIC SAFARI HAT POLY WR
17,sw,39868593,SS22300-WHT-OSFM,6.93254E+12,YTH-TERRY BKT HAT-WHT-OSFM


Do you want to correct the SKUs now? (y/n): y



##################################################################################
##################################################################################

Wrong record with index:  4


,platform,order-id,sku,upc,product-name
4,amazon,111-9337684-4922606,SP21M0042N-BEIL-new,null,KANUT SPORTS Cimarron Safari Hat for Men and W...


...

,SKU,STYLE#,UPC,NAME,COLOR,SIZE,Q,USE,Season,Sex



Suggestion 3 UPC:


,SKU,STYLE#,UPC,NAME,COLOR,SIZE,Q,USE,Season,Sex


##################################################################################

Please enter the correct SKU base on above information: SS22300-WHI	
SKU updated!

Order list updated!


,platform,order-id,sku,upc,product-name,quantity-to-ship,recipient-name
0,amazon,113-5320960-2143433,SP21M0042N-DKL,null,KANUT SPORTS Cimarron Safari Hat for Men and W...,1,Kelley Williams
1,amazon,113-8106292-2154620,SP21M0042N-LGXL,null,KANUT SPORTS Cimarron Safari Hat for Men and W...,1,Steve H. McCombs
2,amazon,111-9337684-4922606,SP21M0042N-LGL,null,KANUT SPORTS Cimarron Safari Hat for Men and W...,1,Linda K. Ferguson
3,amazon,111-9337684-4922606,SP21M0042N-MGM,null,KANUT SPORTS Cimarron Safari Hat for Men and W...,1,Linda K. Ferguson
4,amazon,111-9337684-4922606,SP21M0042N-BEIL,null,KANUT SPORTS Cimarron Safari Hat for Men and W...,1,Linda K. Ferguson
5,amazon,111-9337684-4922606,SP21M0042N-LGM,null,KANUT SPORTS Cimarron Safari Hat for Men and W...,1,Linda K. Ferguson
6,amazon,112-9353466-5926632,SP21M0042N-BEIM,null,KANUT SPORTS Cimarron Safari Hat for Men and W...,1,Rosemary F Roos
7,dsco,88416645,SS22M-08PUTL,6932542845372,CLASSIC SAFARI HAT POLY WR,1,Steve Warren
8,dsco,88429301,SP21M0010-GREL,6932542845792,CLASSIC SAFARI W/DRAPES,1,Michael Mower
9,dsco,88459131,SS22M-09CHOL,6932542845556,CLASSIC SAFARI HAT POLY WR,1,Nicole Hindle


In [13]:
#@title # Summary and Pick Report Generator
#-------------------------------------------------------------------------------refined on 4/10/2024
# Generating Pick List
order_list4 = order_list[(order_list['platform'] == 'amazon') |
                        (order_list['platform'] == 'dsco')    |
                        (order_list['platform'] == 'shopify') |
                        (order_list['platform'] == 'walmart')]
pick_list = order_list4.groupby(['platform', 'sku', 'upc']).sum('quantity-to-ship')
#-------------------------------------------------------------------------------refined on 4/10/2024
# List name format
#today = datetime.datetime.now()
#formatted_date = today.strftime("%m%d%Y")
file_name = f"order_list_{date_now_id()}.xlsx"
file_name2 = f"pick_list_temp_{date_now_id()}.xlsx"

# Saving to local
order_list.to_excel(file_name, index=False)
pick_list.to_excel(file_name2, index = True)

#-------------------------------------------------------------------------------refined on 4/10/2024
#Summary to txt table
#order_list[order_list['platform'] == 'amazon'].groupby('order-id')
a1 = len((order_list[order_list['platform'] == 'amazon']).groupby(['order-id']).count())
a2 = order_list[order_list['platform'] == 'amazon']['quantity-to-ship'].sum()
d1 = len((order_list[order_list['platform'] == 'dsco']).groupby(['order-id']).count())
d2 = order_list[order_list['platform'] == 'dsco']['quantity-to-ship'].sum()
s1 = len((order_list[order_list['platform'] == 'shopify']).groupby(['order-id']).count())
s2 = order_list[order_list['platform'] == 'shopify']['quantity-to-ship'].sum()
w1 = len((order_list[order_list['platform'] == 'walmart']).groupby(['order-id']).count())
w2 = order_list[order_list['platform'] == 'walmart']['quantity-to-ship'].sum()
t1 = len((order_list[order_list['platform'] == 'tsc']).groupby(['order-id']).count())
t2 = order_list[order_list['platform'] == 'tsc']['quantity-to-ship'].sum()
sw1 = len((order_list[order_list['platform'] == 'sw']).groupby(['order-id']).count())
sw2 = order_list[order_list['platform'] == 'sw']['quantity-to-ship'].sum()
spe1 = len((order_list[order_list['platform'] == 'spe']).groupby(['order-id']).count())
spe2 = order_list[order_list['platform'] == 'spe']['quantity-to-ship'].sum()
sam1 = len((order_list[order_list['platform'] == 'sample']).groupby(['order-id']).count())
sam2 = order_list[order_list['platform'] == 'sample']['quantity-to-ship'].sum()

# Example
#+----------+--------+-------+
#| Platform | Orders | Items |
#+==========+========+=======+
#| Amazon   | 16     | 17    |
#+----------+--------+-------+
#| DSCO     | 13     | 17    |
#+----------+--------+-------+
#| Shopify  | 20     | 32    |
#+----------+--------+-------+
#| Walmart  | 1      | 1     |
#+----------+--------+-------+
#| TSC      | 7      | 12    |
#+----------+--------+-------+
#| SW       | 6      | 10    |
#+----------+--------+-------+
#| Total    | 50     | 67    |
#+----------+--------+-------+

t = Texttable()
t.add_rows([['Platform','Orders','Items'], ['Amazon', a1, a2], ['DSCO', d1, d2],['Shopify', s1, s2],['Walmart', w1, w2],['TSC', t1, t2],['SW', sw1, sw2],['SPE', spe1, spe2],['Sample', sam1, sam2], ['Total', a1+d1+s1+w1+t1+sw1+spe1+sam1, a2+d2+s2+w2+t2+sw2+spe2+sam2]])
table_output = t.draw()

file_name2 = f"order_list_{date_now_id()}.txt"
with open(file_name2, 'w') as file:
    file.write(table_output)
print("Order and pick report successfully exported!")

Order and pick report successfully exported!


In [14]:
#@title # SKU Updating
#-------------------------------------------------------------------------------refined on 4/10/2024
# pick list changes
#display(pick_list)

#Add SKU interaction
def add_sku():
   add_df = pd.DataFrame(columns=['Status','SKU','Quantity'])
   while True:
       add_s = input("Do you want to add a SKU? (y/n): ").strip().lower()
       if add_s == 'y':
           sku = input("Enter the SKU: ").strip().upper()
           if sku not in list(inven['SKU']):
                print("The SKU does not exist in our inventory. Please try again.")
                continue
           quantity = input("Enter the adding quantity: ").strip()
           if not quantity.isdigit():
               print("Invalid quantity. Please enter a number.")
               continue
           new_add_df = pd.DataFrame({'Status': 'add', 'SKU': [sku], 'Quantity': [int(quantity)]})
           add_df = pd.concat([add_df, new_add_df], ignore_index=True)
       elif add_s == 'n':
            print('Adding process complete!')
            break
       else:
            print("Invalid response. Please answer 'y' or 'n'.")
   if not add_df.empty:
      display(add_df)
      check = input("Is the above adding summary correct? (y/n): ").strip().lower()
      if check == 'y':
            print('Adding process complete!')
            return add_df
      elif check == 'n':
            return add_sku()
   elif add_df.empty:
        print('There is no adding summary.')
        return add_df
add_df = add_sku()

#-------------------------------------------------------------------------------refined on 4/11/2024
#Remove SKU interaction
def remove_sku():
   remove_df = pd.DataFrame(columns=['Status','SKU','Quantity'])
   while True:
       remove_s = input("Do you want to remove a SKU? (y/n): ").strip().lower()
       if remove_s == 'y':
           sku = input("Enter the SKU: ").strip().upper()
           if (sku not in list(inven['SKU'])) or (sku not in list(order_list['sku'])):
                print("The SKU does not exist in our inventory or order list today, please try again.")
                continue
           quantity = input("Enter the remove quantity: ").strip()
           if not quantity.isdigit():
               print("Invalid quantity. Please enter a number.")
               continue
           new_remove_df = pd.DataFrame({'Status': 'remove', 'SKU': [sku], 'Quantity': [int(quantity)]})
           remove_df = pd.concat([remove_df, new_remove_df], ignore_index=True)
       elif remove_s == 'n':
            print('Remove process complete!')
            break
       else:
            print("Invalid response. Please answer 'y' or 'n'.")
   if not remove_df.empty:
      display(remove_df)
      check = input("Is the above remove summary correct? (y/n): ").strip().lower()
      if check == 'y':
            print('Remove process complete!')
            return remove_df
      elif check == 'n':
            return remove_sku()
   elif remove_df.empty:
        print('There is no remove summary.')
        return remove_df
remove_df = remove_sku()

changes = pd.concat([add_df, remove_df], ignore_index=True)
#display(changes)

Do you want to add a SKU? (y/n): y
Enter the SKU: sp21m0042n-lgm
Enter the adding quantity: 10
Do you want to add a SKU? (y/n): n
Adding process complete!


,Status,SKU,Quantity
0,add,SP21M0042N-LGM,10


Is the above adding summary correct? (y/n): y
Adding process complete!
Do you want to remove a SKU? (y/n): y
Enter the SKU: wp21m0042n-lgl
The SKU does not exist in our inventory or order list today, please try again.
Do you want to remove a SKU? (y/n): sp21m0042n-lgl
Invalid response. Please answer 'y' or 'n'.
Do you want to remove a SKU? (y/n): y
Enter the SKU: sp21m0042n-lgl
Enter the remove quantity: 1
Do you want to remove a SKU? (y/n): n
Remove process complete!


,Status,SKU,Quantity
0,remove,SP21M0042N-LGL,1


Is the above remove summary correct? (y/n): y
Remove process complete!


In [15]:
#@title # Order Updating
#-------------------------------------------------------------------------------refined on 4/11/2024
original_order = (order_list[['sku','quantity-to-ship']]).groupby(['sku']).sum(['quantity-to-ship']).reset_index()
#Updating to original order
for index, row in changes.iterrows():
    sku = row['SKU']
    quantity_change = row['Quantity']
    status = row['Status']

    # Find the row in DataFrame original_order that matches the SKU
    if sku in original_order['sku'].values:
        original_order_index = original_order[original_order['sku'] == sku].index[0]

        # Update the quantity based on the status
        if status == 'add':
            original_order.at[original_order_index, 'quantity-to-ship'] += quantity_change
        elif status == 'remove':
            original_order.at[original_order_index, 'quantity-to-ship'] -= quantity_change
    elif sku not in original_order['sku'].values and status == 'add':
        new_sku = pd.DataFrame({'sku': [sku], 'quantity-to-ship': [quantity_change]})
        original_order = pd.concat([original_order, new_sku], ignore_index=True)

# Checking orders with negative quantity
if (original_order['quantity-to-ship'] < 0).any():
    print("Warning: Negative quantities found in the original order, please re-update the SKUs.")
else:
    print('Please check the final order summary for today:')
    display(original_order)

#-------------------------------------------------------------------------------refined on 4/11/2024
    export_order = input("Do you want to export the final order summary for today? (y/n): ").strip().lower()
    if export_order == 'y':
        file_name3 = f"order_summary_{formatted_date}.xlsx"
        original_order.to_excel(file_name3, index=False)
        print('Updated order summary exported!')
    elif export_order == 'n':
        print('Order summary updated!')
    else:
        print("Invalid response. Please answer 'y' or 'n'.")

Please check the final order summary for today:


,sku,quantity-to-ship
0,10577-GREXL,1
1,74535-BEI,1
2,74535-GRE,1
3,FW23M0072-BLAXL,1
4,SP20M079-CAML,3
5,SP20M079-CAMM,4
6,SP20M079-CAMXL,3
7,SP20M079-MOSL,3
8,SP20M079-MOSM,3
9,SP20M079-MOSXL,3


Do you want to export the final order summary for today? (y/n): n
Order summary updated!


In [16]:
#@title # Inventory Updating
#-------------------------------------------------------------------------------refined on 4/11/2024
#Updating Inventory
for index, row in original_order.iterrows():
    sku = row['sku']
    quantity_to_ship = row['quantity-to-ship']

    # Find the corresponding SKU in the inventory and update quantity
    if sku in inven['SKU'].values:
        inven_index = inven[inven['SKU'] == sku].index[0]
        inven.at[inven_index, 'Q'] -= quantity_to_ship
    else:
        print("Warning: The SKU does not exist in our inventory, please re-update the SKUs.")
#print("Inventory updated successfully, please check the updated inventory for today:")
# add not use
new_inven = pd.concat([inven, inven_0], ignore_index=True)

file_name4 = f"inventory_{date_now_id()}.xlsx"
new_inven.to_excel(file_name4, index=False)
print('Updated inventory exported!')
#-------------------------------------------------------------------------------refined on 4/11/2024
# export_inventory = input("Do you want to export the inventory for today? (y/n): ").strip().lower()
# if export_inventory == 'y':
#     file_name4 = f"inventory_{date_now_id()}.xlsx"
#     new_inven.to_excel(file_name4, index=False)
#     print('Updated inventory exported!')
# elif export_order == 'n':
#     print('Inventory not exported!')
# else:
#     print("Invalid response. Please answer 'y' or 'n'.")

Updated inventory exported!
